We use the newely tarined embeddings for word2vec transformation

There is a change in preprocessing in this notebook, we removed all the numbers from tweets, which helped in training more robust word2vec embeddings

Analysis on scraped dataset

### Import

In [0]:
import pandas as pd
import numpy as np

In [0]:
import pickle
import sys
import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cf_data_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NAACL_SRW.csv',encoding = "ISO-8859-1")
cf_data_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NLP+CSS.csv')

cf_data_3 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_2/labeled_data.csv',encoding = "ISO-8859-1")
## this is the scraped data

cf_data_3.rename({'Unnamed: 0':'ID','tweet':'Tweets'},axis=1,inplace=True)

labels_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NAACL_SRW_2016.csv',header=None,names=['ID','class'])
labels_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NLP+CSS_2016.csv',sep='\s')

labels_2.rename({'TweetID':'ID','Expert':'class'},axis=1,inplace=True)

cf_data_1.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)
cf_data_2.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


### Function for merging

In [0]:
def label_merging(data, labels):
    labels['ID'] = labels['ID'].astype(int)
    print(labels['ID'].nunique())
    print('Null IDs in data 1 = ' ,data['ID'].isna().sum())
    
    data['ID'].fillna(0,inplace=True)
    data['ID'] = data['ID'].astype(int)
    
    print('data shape ='  ,data.shape)
    print('IDs common in data and labels =',sum(data['ID'].isin(labels['ID'])))
    
    train = data.merge(labels, on='ID',how='inner')#['class'].isna().sum()
    return train

In [6]:
train_1 = label_merging(cf_data_1,labels_1)

train_2 = label_merging(cf_data_2, labels_2)

train_3 = cf_data_3.copy()

16849
Null IDs in data 1 =  2
data shape = (16037, 11)
IDs common in data and labels = 11238
6909
Null IDs in data 1 =  0
data shape = (6271, 11)
IDs common in data and labels = 6271


In [0]:
t1 = train_1[['ID','Tweets','class']]
t2 = train_2[['ID','Tweets','class']]
t3 = train_3[['ID','Tweets','class']]
merged = pd.concat([t1,t2,t3],axis=0).reset_index(drop=True)

### Target Analysis

### Basic Preprocessing

In [0]:
train = merged.copy()

train.rename(columns={'Tweets':'tweet'},inplace=True)
train['tweet'] = train['tweet'].astype(str)

Every word followed by @ is some twitter ID of an user, which shouldn't be considered in our analysis, so lets do the stemming, where we remove @ alonwith the word followed by it

In [9]:
train['tweet'] = train['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('@')==-1]))
train['tweet'] = train['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('http')==-1]))
train['tweet'] = train['tweet'].apply(lambda x:''.join([i for i in x if not i.isdigit()]))
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

remove_word = ['rt','mkr','im']
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in remove_word))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Doesnt really make sense to remove rare words, i.e. the words with count 1. Because we might lose hateful words this way

In [10]:
from textblob import TextBlob
nltk.download('punkt')

from textblob import Word
nltk.download('wordnet')
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['tweet'].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0    entire concept god corresponds tyrannical eart...
1    prophet rapist murderer pedophile caravan robb...
2    yazidi child taken parent forcibly converted i...
3    girl equivalent irritating asian girl couple y...
4    love islamofascists recruit year old jihadis t...
Name: tweet, dtype: object

### Target creation

In [11]:
train['class'].unique()#.isna().sum()

array(['racism', 'none', 'sexism', 'neither', 'both', 2, 1, 0],
      dtype=object)

In [12]:
train['class'].replace(['racism', 'sexism',0, 1, 'both', 'none', 'neither',2],['hate','hate','hate','hate','hate','null','null','null'],inplace=True)
train['class'].value_counts()

hate    24942
null    17422
Name: class, dtype: int64

In [0]:
train['class'].replace(['null','hate'],[0,1],inplace=True)

### Removing duplicates

In [0]:
# hate_text = train[train['class']==1]['tweet']
# null_text = train[train['class']==0]['tweet']

In [0]:
# hate_text.to_csv(r'hate_speech.txt', header=None, index=None, sep=' ')

In [0]:
# null_text.to_csv(r'null_speech.txt', header=None, index=None, sep=' ')

In [17]:
sum(train['ID'].value_counts()>1)

2760

In [18]:
train.shape

(42364, 3)

In [0]:
train = train.drop_duplicates(subset='ID')

### Basic Model on whole dataset

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))

tfidf.fit(train['tweet'])

x_t = tfidf.transform(x_train)
x_v = tfidf.transform(x_valid)


In [0]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    if task=='binary:logistic':
      print('validation accuracy', clf.score(x_v,y_v))
      print('validation f1_score',f1_score(clf.predict(x_v),y_v))
      print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
      print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    if task=='reg:linear':
        if x_v!=None:
            print('validation r2_score', clf.score(x_v,y_v))
            print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [0]:
# y_t.value_counts()

# 19794/11810

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score
from xgboost import XGBClassifier

lgr =  LogisticRegression(n_jobs=1)
# model_training(lgr,x_t,y_t,x_v,y_v)

xgb = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.4266790777602751)
# xgb = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.6760372565622355)
model_training(xgb,x_t,y_t,x_v,y_v)


training accuracy 0.8751740286039742
validation accuracy 0.8567451278157429
validation f1_score 0.8815403934700711
validation roc_auc score 0.9185450724629186
confusion matrix 
 [[2558  339]
 [ 793 4212]]


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1.4266790777602751,
              seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
# results with LogisticRegression(n_jobs=1)
# training accuracy 0.857264903176813
# validation accuracy 0.8553530751708428
# validation f1_score 0.8783654357773757
# validation roc_auc score 0.9169276701896655
# confusion matrix 
#  [[2632  265]
#  [ 878 4127]]

In [0]:
# XGB Results
# training accuracy 0.8751740286039742
# validation accuracy 0.8567451278157429
# validation f1_score 0.8815403934700711
# validation roc_auc score 0.9185450724629186
# [[2558  339]
#  [ 793 4212]]

In [0]:
#  this was on different split, as we had duplicates included that time
# training accuracy 0.8751290903189637
# validation accuracy 0.854242889177387
# validation f1_score 0.8701503522237408
# validation roc_auc score 0.9160894624941864
# confusion matrix 
#  [[3100  356]
#  [ 879 4138]]
# XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0,
#               learning_rate=0.1, max_delta_step=0, max_depth=5,
#               min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1.4266790777602751,
#               seed=None, silent=None, subsample=1, verbosity=1)

### Training on new word2vec model

In [29]:
import gensim
import logging
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/model_transfer_learning.txt", binary=False)
# Above is domain + pretrained embeddings

# wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/GoogleNews-vectors-negative300.bin.gz", binary=True)
# above is only pretrained embeddings

wv.init_sims(replace=True)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [31]:
from sklearn.model_selection import train_test_split

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train_w2v, test_w2v = train_test_split(train, test_size=0.2, random_state = 42)
# x_t,x_v,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

test_tokenized = test_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values
train_tokenized = train_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score

In [0]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    if task=='binary:logistic':
      print('validation accuracy', clf.score(x_v,y_v))
      print('validation f1_score',f1_score(clf.predict(x_v),y_v))
      print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
      print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    if task=='reg:linear':
      print('validation r2_score', clf.score(x_v,y_v))
      print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [38]:
%%time
xgb_w2v = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.4266790777602751)
lgr_w2v = LogisticRegression(n_jobs=1)

lgbm_w2v = LGBMClassifier(n_estimators=500)

gbdt_w2v = GradientBoostingClassifier(n_estimators=500)

# model_training(xgb_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])

# model_training(lgr_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])

# model_training(lgbm_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])

model_training(gbdt_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])


training accuracy 0.9290596127072522
validation accuracy 0.8716780561883067
validation f1_score 0.896804396499084
validation roc_auc score 0.9450967057460726
confusion matrix 
 [[2482  450]
 [ 564 4406]]
CPU times: user 8min 47s, sys: 337 ms, total: 8min 48s
Wall time: 8min 48s


In [0]:

# Logistic Regression LogisticRegression(n_jobs=1)  Domain + pretrained EMbeddings
# training accuracy 0.8715668902670548
# validation accuracy 0.8672487977727158
# validation f1_score 0.8934051417538867
# validation roc_auc score 0.9390565768416776
# confusion matrix 
#  [[2457  475]
#  [ 574 4396]]

# XGBoost Domain + Pretrained Embeddings
# training accuracy 0.9944310846728263
# validation accuracy 0.8772462667679068
# validation f1_score 0.9021190716448032
# validation roc_auc score 0.950407012333208
# confusion matrix 
#  [[2462  470]
#  [ 500 4470]]


# Logistic Regression LogisticRegression(n_jobs=1)  Only Pretrained EMbeddings
# training accuracy 0.8596696620680927
# validation accuracy 0.8535813718046065
# validation f1_score 0.8819749056411302
# validation roc_auc score 0.9265340679822454
# confusion matrix 
#  [[2422  510]
#  [ 647 4323]]
# CPU times: user 2.16 s, sys: 162 ms, total: 2.33 s
# Wall time: 2.28 s

# XGBoost + Pretrained Embeddings
# training accuracy 0.9898114162764207
# validation accuracy 0.8599088838268792
# validation f1_score 0.8885084097089335
# validation roc_auc score 0.935837432507734
# confusion matrix 
#  [[2384  548]
#  [ 559 4411]]


In [0]:
# # GBDT + Word2Vec (domain+pretrained)
# training accuracy 0.9290596127072522
# validation accuracy 0.8716780561883067
# validation f1_score 0.896804396499084
# validation roc_auc score 0.9450967057460726
# confusion matrix 
#  [[2482  450]
#  [ 564 4406]]

In [0]:
print('validation accuracy', xgb_w2v.score(X_test_word_average,test_w2v['class']))
print('validation f1_score',f1_score(xgb_w2v.predict(X_test_word_average),test_w2v['class']))

validation accuracy 0.8772462667679068
validation f1_score 0.9021190716448032


In [0]:
confusion_matrix(test_w2v['class'],xgb_w2v.predict(X_test_word_average))

array([[2462,  470],
       [ 500, 4470]])

In [0]:
tuned_pred = (xgb_w2v.predict_proba(X_test_word_average)[::,-1]>0.3).astype(int)
confusion_matrix(test_w2v['class'],tuned_pred)

array([[2148,  784],
       [ 295, 4675]])

In [0]:
f1_score(test_w2v['class'],tuned_pred)

0.8965384984178733

In [0]:
test_w2v['class'].value_counts()

1    4970
0    2932
Name: class, dtype: int64

In [0]:
pd.set_option('display.max_colwidth', -1)

In [0]:
check_df = test_w2v#[test_w2v['class']!=xgb_w2v.predict(X_test_word_average)]
check_df['preds'] = xgb_w2v.predict(X_test_word_average)
check_df['hate_probab'] = xgb_w2v.predict_proba(X_test_word_average)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
preds_analysis = merged.drop_duplicates(subset='ID').drop('class',axis=1).merge(check_df, on='ID',how='inner')

In [0]:
preds_analysis

,ID,Tweets,tweet,class,preds,hate_probab
0,551769698917167104,"@anjemchoudary Your prophet was a rapist, murderer, pedophile, caravan robber, slave trader, bigot and sexist. God would never use the scum.",prophet rapist murderer pedophile caravan robber slave trader bigot sexist god would never use scum,1,1,0.960933
1,551659627872415744,@lauracdean I love how the Islamofascists recruit 14 and 15 year old jihadis and then talk about minors in reference to 17 year olds.,love islamofascists recruit year old jihadis talk minor reference year old,1,1,0.646757
2,552530506756423680,"RT @jemfleming: @riwired @GOP @VileIslam 'When the Muslim population in a region reaches a certain.. point, the agitation begins..' http://",muslim population region reach certain point agitation begin,1,0,0.168379
3,552534253976969216,@HillaryGuess @riwired @izrinhariri Why are you flapping your lips. None of this is related to the barbarity of Islam?,flapping lip none related barbarity islam,1,0,0.483724
4,552178267927883776,Imam's who disagree with Jihad are attacked by Muslims. So where are these moderate Muslims? #Islam http://t.co/hkh5tyEIej,imam disagree jihad attacked muslim moderate muslim islam,1,1,0.949111
5,552533207426797568,@HillaryGuess @riwired @izrinhariri @AtharHKhan Islam attacked and conquered 2/3 of Christianity before there were any crusades.,islam attacked conquered christianity crusade,1,1,0.905910
6,551935344644349952,RT @Mola_Tiz: Palestinians accepted the 2state solution.one for Hamas and one for ISIS. http://t.co/kPVBxc6MJx,palestinian accepted state solutionone hamas one isi,1,0,0.272181
7,552534024334614528,@HillaryGuess @riwired @izrinhariri @AtharHKhan Now you see ISIS following the Quran and Hadiths and prophet exactly in their genocide.,see isi following quran hadith prophet exactly genocide,1,1,0.732003
8,551812016768172032,RT @IAMMGraham: I've been called an Islamophobe for saying Islam needs a theological reformation. Now the head of Egypt agrees:\r\nhttp://t.co,ive called islamophobe saying islam need theological reformation head egypt agrees,1,1,0.699337
9,553017226639704064,@LUVSHINECAFE @bradybaggett We need to outlaw Islam outright.,need outlaw islam outright,1,1,0.513906


In [0]:
preds_analysis.to_csv('test_predictions.csv',index=True,header=True)

In [0]:
# testing on sample sentences
check = pd.DataFrame(columns=['tweet'])

for i in range(0,5):
  a = input()
  check.loc[i, 'tweet'] = a

u bitch should stay inside
nigga got no chill
islam is not bad
i dont think all muslims are terrorists
donald trump is a good guy


In [0]:
check_tokenized = check.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

check_word_average = word_averaging_list(wv,check_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
xgb_w2v.predict(check_word_average)

array([1, 1, 1, 0, 0])

In [0]:
xgb_w2v.predict_proba(check_word_average)

array([[2.5337934e-04, 9.9974662e-01],
       [2.3335320e-01, 7.6664680e-01],
       [2.6771992e-01, 7.3228008e-01],
       [9.7094500e-01, 2.9054986e-02],
       [9.4118708e-01, 5.8812920e-02]], dtype=float32)

### Doc2Vec

In [0]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

In [0]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
  
  
train_doc2v, test_doc2v = train_test_split(train, test_size=0.2, random_state = 42)

  
train_tagged = train_doc2v.apply(lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r['class']]), axis=1)
test_tagged = test_doc2v.apply(lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r['class']]), axis=1)

In [0]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [0]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 33891/33891 [00:00<00:00, 1426942.49it/s]


In [0]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 33891/33891 [00:00<00:00, 1893907.97it/s]


CPU times: user 1min 8s, sys: 12.5 s, total: 1min 21s
Wall time: 52.9 s


In [0]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [0]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [0]:
%%time

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Testing accuracy 0.6468783193674024
Testing F1 score: 0.6362708385263394
CPU times: user 3.14 s, sys: 128 ms, total: 3.27 s
Wall time: 3.24 s


In [0]:
# # Testing accuracy 0.7482591762067745
# # Testing F1 score: 0.7487943851129049
# doc2vec accuracy with dm=1

In [0]:
# Testing accuracy 0.8671072819544435
# Testing F1 score: 0.8675608637800902
# doc2vec accuracy with dm=0. means CBOW kind of approach

### Lets try XGBoost

In [0]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [0]:
%%time
xgb_doc2v = XGBClassifier(n_estimators=3000, max_depth=8,learning_rate=0.1,scale_pos_weight=1.4266790777602751)

xgb_doc2v.fit(X_train, y_train)


CPU times: user 1h 36min 49s, sys: 1.31 s, total: 1h 36min 50s
Wall time: 1h 36min 52s


In [0]:
xgb_doc2v.score(X_train, y_train)

0.999793455489658

In [0]:
y_pred = xgb_doc2v.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6645816121798654
Testing F1 score: 0.6556511542024396


In [0]:
# Testing accuracy 0.6873598489319014
# Testing F1 score: 0.6705179542826458
# XGB n_est = 500, max_depth=5